In [727]:
import logging
import os
import openai
import itertools

from typing import List

from actionweaver.llms.azure.chat import ChatCompletion
from actionweaver.llms.azure.tokens import TokenUsageTracker
from actionweaver import action, SelectOne, RequireNext

from actionweaver.llms.openai.chat import OpenAIChatCompletion
from actionweaver.actions.factories.pydantic_model_to_action import action_from_model
from actionweaver import action
from actionweaver.mixins.examples import LangChainTools, Folium, OpenAIAPI
openai.api_key = os.getenv("OPENAI_API_KEY")

In [728]:
logging.basicConfig(
    filename='planning.log',
    filemode='a',
    format='%(asctime)s.%(msecs)04d %(levelname)s {%(module)s} [%(funcName)s] %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

In [729]:
 

def print_output(output):
    from collections.abc import Iterable
    if isinstance(output, str):
        print (output)
    elif isinstance(output, Iterable):
        for chunk in output:
            content = chunk.choices[0].delta.content
            if content is not None:
                print(content, end='')


In [730]:


class Task(BaseModel):
    id: int
    objective: str
    thought: str
    action: str
    action_input: str
    dependencies: List[int] = []


class Plan(BaseModel):
    problem: str
    tasks: List[Task]

    def visualize(self):
        # pyvis 0.3.1
        from pyvis.network import Network
        from IPython.display import display, HTML

        net =  Network(notebook=True, directed=True)    
        for task in self.tasks:
            net.add_node(task.id, label=task.objective)
            for dep in task.dependencies:
                net.add_edge(dep, task.id)
        return net

In [684]:
class ActionMixin:  
    @action(name="ExecutePythonCode")
    def interpret_python_code(self, code_text: str):
        """
        Execute Python code text and returns the result.
    
        Args:
            code_text (str): A string containing self containing Python code to be interpreted.
    
        Example:
            >>> code_to_interpret = "2 + 3"
            >>> result = interpret_python_code(code_to_interpret)
            >>> print(result)
            5
        """
        import sys
        from io import StringIO
        # Create a local dictionary for the execution environment.
        env = {}
        
        try:
            # Redirect stdout to capture the output.
            original_stdout = sys.stdout
            sys.stdout = captured_output = StringIO()
            
            # Execute the code in the provided environment.
            exec(code_text, env)
            
            # Retrieve the last expression's value in the environment.
            result = env.get('__builtins__', {}).get('None', None)
            
            return {"return": result, "stdout": captured_output.getvalue()}
        except Exception as e:
            return f"Error: {str(e)}", captured_output.getvalue()
        finally:
            # Restore the original stdout.
            sys.stdout = original_stdout

    @action(name="GoogleSearch")
    def search(self, query: str):
        """
        Perform a Google search and return query results with titles and links.
    
        :param query: The search query to be used for the Google search.
        """
        from langchain.utilities import GoogleSearchAPIWrapper
    
        search = GoogleSearchAPIWrapper()
        res = search.results(query, 10)
        return res

In [767]:
class Agent(ActionMixin):
    """
    An agent that plans ahead and then executes actions based on its plan.
    """

    
    def __init__(self, logger):
        self.logger = logger
        # self.chat = ChatCompletion(
        #             model="gpt-35-turbo-0613-16k", 
        #             azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
        #             api_key=os.getenv("AZURE_OPENAI_KEY"),  
        #             api_version="2023-10-01-preview", 
        #             logger=logger)


        self.chat = OpenAIChatCompletion("gpt-4-0613",  logger=logger)

        self.system_message = {"role": "system", "content": "You are a resourceful assistant. Consider using PlanAndExecute if you're unsure about your ability to provide a satisfactory response right away."}
        
        self.messages = [self.system_message]
        self.plans = []

    def __call__(self, text):
        self.messages += [{"role": "user", "content":text}]
        # response = self.plan_and_execute.invoke(chat, self.messages)

        response = self.plan_and_execute.invoke(chat, self.messages, stream=True)
        # response = self.chat.create(messages=self.messages, actions =[self.plan_and_execute], stream=True)
        return response


    def create_plan(self, text) -> Plan:
        self.messages += [{"role": "user", "content":text}]
        response = action_from_model(Plan).invoke(chat, self.messages, stream=False)
        return response


    @action("PlanAndExecute")
    def plan_and_execute(self, plan: Plan):
        """
        Create and then executes a given plan.
    
        This function takes a 'Plan' object as input and executes it. The 'Plan' should
        contain a sequence of tasks to be performed in a specific order.

        Parameters:
        plan (Plan): A 'Plan' object representing the sequence of actions to be executed.
        """
        # ANSI color codes
        RESET = "\033[0m"
        RED = "\033[91m"
        GREEN = "\033[92m"
        BLUE = "\033[94m"
        MAGENTA ="\033[95m"
        
        # Text to print
        OBSERVATION = f"{RED}Observation{RESET}"
        OBJECTIVE = f"{GREEN}Objective{RESET}"
        THOUGHT = f"{BLUE}Thought{RESET}"
        ACTION = f"{MAGENTA}Action{RESET}"
        ACTION_INPUT = f"{MAGENTA}ActionInput{RESET}"

        
        
        plan = Plan.parse_obj(plan)
        self.plans += [plan]


        # Execute the task graph using topo sort, starting with tasks with no dependencies
        results = {}        
        indegrees = {}
        out = {}
        tasks = {}        
        to_be_executed = []
        
        for task in plan.tasks:
            out[task.id] = []
            
        for task in plan.tasks:
            tasks[task.id] = task

            indegrees[task.id] = len(task.dependencies)
            for dep in task.dependencies:
                out[dep].append(task.id)

            if indegrees[task.id] == 0:
                to_be_executed.append(task.id)

        last_executed_tasks = [] # Tasks executed in last iteration
        while len(to_be_executed) > 0:
            buf = []
            
            for t in to_be_executed:
                print ('#' * 100 + "\n")

                context = []
                for dep in tasks[t].dependencies:
                    context += [f"{OBSERVATION} : {results[dep]}"]
                context = '\n'.join(context)
            
                message = f"{context}\n{OBJECTIVE}: {tasks[t].objective}\n{THOUGHT}: {tasks[t].thought}\n{ACTION}: {tasks[t].action}\n{ACTION_INPUT}: {tasks[t].action_input}"

                print (message + "\n")
                
                # execute 
                res = self.chat.create(messages=[self.system_message, {"role": "user", "content":message}], actions =[self.interpret_python_code, self.search], stream=False)
                results[t] = res

                print (f"Result: {res}")
                print ('#' * 100 + "\n")
                
                last_executed_tasks += [t]

                for d in out[t]:
                    indegrees[d] -= 1
                    if indegrees[d] == 0:
                        buf.append(d)

            to_be_executed = buf
        return '\n'.join([results[t] for t in last_executed_tasks])
    


In [774]:
question = """who are mayors of top 3 populated cities in USA """

In [775]:
agent = Agent(logger)
plan = agent.create_plan(f"""PlanAndExecute: {question}""")

In [785]:
plan.visualize().show('nx.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [777]:
res = agent.plan_and_execute(plan)

/var/folders/rk/nvn6c0b92tb48wzzf6k_8b380000gn/T/ipykernel_18621/3159929866.py:61: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.2/migration/
  plan = Plan.parse_obj(plan)


####################################################################################################


Objective: Research the top 3 populated cities in the USA
Thought: I can search for the current population data of cities in the USA
Action: Search for the current population data of cities in the USA
ActionInput: USA cities population data

Result: The top three most populated cities in the USA are:

1. New York - 8,550,405
2. Los Angeles - 3,971,883
3. Chicago - 2,720,546

These numbers are based on the data available at the time of the search.
####################################################################################################

####################################################################################################

Observation : The top three most populated cities in the USA are:

1. New York - 8,550,405
2. Los Angeles - 3,971,883
3. Chicago - 2,720,546

These numbers are based on the data available at the time of the search.
Objective: Identify the may

In [779]:
res

"The top three most populated cities in the USA are:\n\n1. New York - 8,550,405\n2. Los Angeles - 3,971,883\n3. Chicago - 2,720,546\n\nThese numbers are based on the data available at the time of the search.\nThe current mayor of New York is Eric Adams. Now, let's find out the mayors of Los Angeles and Chicago.\nThe current mayor of Los Angeles is Karen Bass. Now, let's find out the mayor of Chicago."

In [782]:
TASK_PROMPT = """Use following format to reason:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of {tool_names}
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
"""


class Task(BaseModel):
    question: str
    thought: str
    action: str
    action_input: str


class ReActAgent(ActionMixin):
    """ 
    Agent to implement the ReAct logic.
    https://react-lm.github.io/
    """

    
    SYS_PROMPT = """You are a resourceful assistant."""

    CONTEXT = """
----------------------
{tasks}

----------------------
Answer the question as best you can, use the context above and continue with the reasoning process.
Only give the final answer when you're confident, otherwise continue the reasoning.
"""

    
    def __init__(self, logger, execute=False):
        self.logger = logger
        self.chat = ChatCompletion(
                    model="gpt-35-turbo-0613-16k", 
                    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
                    api_key=os.getenv("AZURE_OPENAI_KEY"),  
                    api_version="2023-10-01-preview", 
                    logger=logger)


        # self.chat = OpenAIChatCompletion("gpt-4-0613",  logger=logger)
        self.system_message = {"role": "system", "content": self.SYS_PROMPT}
        self.msgs = []
        self.tasks = []

    def __call__(self, text):
        self.tasks = []
        max_iters = 5
        actions = {self.search.name: self.search}
        
        while max_iters:
            system_message = [{"role": "system", "content": self.SYS_PROMPT}]

            ans = self.chat.create(messages=system_message + [{"role": "user", "content": self.CONTEXT.format(tasks = text + '\n'.join(self.tasks)) + text}], actions =[action_from_model(Task, description=TASK_PROMPT.format(tool_names=list(actions.keys())))], stream=False)
            if isinstance(ans, Task):
                obs = actions[ans.action](ans.action_input)
                
                self.tasks.append(f"\nThought:{ans.thought}\nAction:{ans.action}\nActionInput:{ans.action_input}\nObservation:{obs}\n")
            else:
                return ans

            
            max_iters -= 1

        return "Stopped!"


In [784]:
agent = ReActAgent(logger)
ans = agent("""who are mayors of  top 3 most populated cities in the USA""")